In [ ]:
Projet

In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [44]:
headers= {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    
}
food_recipe = []

links = []

for  x in range(1,4):
    page = requests.get(f'https://www.marmiton.org/recettes?type=platprincipal&page={x}')
    soup = bs(page.text, "lxml")
    
    homePages = soup.find_all("div",{"class":"recipe-card"})
    for homePage in homePages:
        href_tags = homePage.find("a").get('href')
    
        links.append(href_tags)
    
print(len(links))
    

90


# Extract recipe informations

In [48]:
data = []
for link in links:
#url = 'https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'
    res = requests.get(link, headers=headers)
    soup = bs(res.content, 'lxml')

    title = soup.find('h1', class_="main-title show-more").text.strip()


    items = soup.find_all('span', class_="ingredient-name show-icon")
    ingredient = []
    for i in items:
        ingredient.append(i.getText().strip())

    steps = []  
    Cooking_steps = soup.find_all('div', class_="recipe-step-list__container")
    for i in Cooking_steps:
        steps.append(i.getText().strip())

    recipe = {
        'title':title,
        'ingredient':ingredient,
        'steps':steps
    }
    #print(recipe)


In [49]:
def clean_string(var):
    var = str(var)
    var =var.rstrip()
    var = var.replace('\n','')
    var =var.replace("'\'", '')
    return var

recipe['steps'] = clean_string(steps)


In [50]:
print(recipe)

{'title': 'Porc caramel', 'ingredient': ['eau', 'sucre', 'gingembre', 'cube de bouillon', 'filet mignon', 'eau', 'mélange 4 épices', 'sauce soja', 'oignon jaune'], 'steps': '[\'Étape 1\\n\\n\\n\\n \\n\\n\\nDécouper le porc en bouchées.\', "Étape 2\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n \\n\\n\\nFaire chauffer 50 cl d\'eau. Ajouter les deux cubes de bouillon de poule, 1 cuillère à soupe de gingembre moulu, 1 cuillère à soupe de mélange quatre épices et 3 cuillères à soupe de sauce soja.", "Étape 3\\n\\n\\n\\n\\n\\n \\n\\n\\nDans une sauteuse, préparez votre caramel avec le sucre et l\'eau.", \'Étape 4\\n\\n\\nUne fois le caramel prêt, ajouter le bouillon et tourner très vite pour faire dissoudre le caramel (qui va se durcir) dans le bouillon.\', "Étape 5\\n\\n\\n\\n \\n\\n\\nUne fois le caramel dissout, rajouter la viande et l\'oignon coupé en gros morceaux et mettre à feu très vif. Laisser réduire jusqu\'à ce que tout le liquide se soit évaporé (environ 25 min) et que la viande se mêle au

In [7]:
import json
print(json.dumps(recipe, indent=2))

{
  "title": "Tzatziki (Gr\u00e8ce)",
  "ingrediens": [
    "sel",
    "poivre",
    "huile d'olive",
    "ail",
    "concombre",
    "menthe",
    "yaourt"
  ],
  "steps": [
    "\u00c9tape 1\n\n\nR\u00e2per le concombre et le faire d\u00e9gorger 15 min.",
    "\u00c9tape 2\n\n\nLe mettre dans un saladier, ajouter l'ail \u00e9cras\u00e9, la menthe et le coriandre finement hach\u00e9s, mettre le yaourt, l'huile, le sel et le poivre, remuer et mettre au frais.",
    "\u00c9tape 3\n\n\nLe servir tr\u00e8s frais avec du pain grec en ap\u00e9ritif, ou tel quel en entr\u00e9e."
  ]
}


In [ ]:
from pymongo import MongoClient

myclient = MongoClient("mongodb://localhost:27017")
db = myclient["projetScrap"]
collection = db['projetESGI']

collection.insert_many(recipe)
